In [28]:
import pandas as pd
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from sklearn.metrics import mean_squared_error, r2_score, balanced_accuracy_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import random
from collections import defaultdict
import pickle
import seaborn as sns
from scipy.stats import rankdata

In [25]:
with open('perfect_regr_models.pickle', 'rb') as f:
    perfect_model_regr = pickle.load(f)
with open('perfect_class_models.pickle', 'rb') as f:
    perfect_model_class = pickle.load(f)    
 

In [29]:
def search_of_max_pki_in_dataset_class(dataset, perfect_model):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_train_ex_regr, y_train_ex, y_test_ex = data[0], data[1], data[2], data[4], data[5]
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    while all(x in x_test_in.index for x in perfect_model[dataset][1]) != True:
        x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    max_pki_index = y_train_ex_regr[['pKi']].idxmax()[0]
    max_pki = y_train_ex_regr[['pKi']].max()[0]
    best_points = perfect_model[dataset][1][:]
    if max_pki_index not in best_points:
        iteration_of_max_pki = 'max pki is not in selected points'
    else:
        if max_pki_index in y_train_in.index:
            random_point = random.choice(list(x_test_in.index))
            x_test_in, y_test_in = x_test_in.append(x_train_in.loc[max_pki_index]), y_test_in.append(y_train_in.loc[max_pki_index])
            x_train_in, y_train_in = x_train_in.append(x_test_in.loc[random_point]), y_train_in.append(y_test_in.loc[random_point])              
            x_train_in, y_train_in = x_train_in.drop(max_pki_index), y_train_in.drop(max_pki_index)
            x_test_in, y_test_in = x_test_in.drop(random_point), y_test_in.drop(random_point)        

        iteration = 0
        while True:

            adding_points = best_points[:5]
            del best_points[:5]
            print(adding_points)       
            for point in adding_points:
                x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
                x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)
            iteration += 1
            if max_pki_index in adding_points:
                iteration_of_max_pki = iteration
                break

    return iteration_of_max_pki, max_pki

In [30]:
class_search = {}
for dataset in tqdm(perfect_model_class.keys()):
    class_search[dataset] = search_of_max_pki_in_dataset_class(dataset, perfect_model_class)

  6%|▌         | 1/17 [00:02<00:37,  2.36s/it]

[88, 350, 10, 301, 236]
[184, 128, 43, 204, 134]
[212, 324, 16, 19, 335]
[152, 187, 313, 209, 122]


 12%|█▏        | 2/17 [00:03<00:27,  1.85s/it]

[274, 134, 433, 43, 58]
[308, 212, 191, 34, 257]
[306, 316, 50, 379, 190]


 18%|█▊        | 3/17 [00:03<00:21,  1.52s/it]

...

 88%|████████▊ | 15/17 [00:22<00:06,  3.46s/it]

[388, 393, 177, 256, 187]
[119, 129, 38, 384, 147]
[138, 171, 361, 141, 154]


 94%|█████████▍| 16/17 [00:28<00:04,  4.35s/it]

[337, 185, 26, 307, 248]
[103, 232, 247, 324, 129]
[239, 65, 95, 56, 253]
[241, 206, 219, 204, 155]
[40, 308, 238, 169, 115]
[188, 176, 269, 201, 279]


100%|██████████| 17/17 [00:29<00:00,  1.71s/it]


In [46]:
def search_of_max_pki_in_dataset_regr(dataset, perfect_model):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_train_ex, y_test_ex = data[0], data[1], data[2], data[3]
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    while all(x in x_test_in.index for x in perfect_model[dataset][1]) != True:
        x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    max_pki_index = y_train_ex[['pKi']].idxmax()[0]
    max_pki = y_train_ex[['pKi']].max()[0]
    best_points = perfect_model[dataset][1][:]
    if max_pki_index not in best_points:
        iteration_of_max_pki = 'max pki is not in selected points'
    else:

        if max_pki_index in y_train_in.index:
            random_point = random.choice(list(x_test_in.index))
            x_test_in, y_test_in = x_test_in.append(x_train_in.loc[max_pki_index]), y_test_in.append(y_train_in.loc[max_pki_index])
            x_train_in, y_train_in = x_train_in.append(x_test_in.loc[random_point]), y_train_in.append(y_test_in.loc[random_point])              
            x_train_in, y_train_in = x_train_in.drop(max_pki_index), y_train_in.drop(max_pki_index)
            x_test_in, y_test_in = x_test_in.drop(random_point), y_test_in.drop(random_point)        

        iteration = 0
        while True:

            adding_points = best_points[:5]
            del best_points[:5]
            #print(adding_points)

            for point in adding_points:
                x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
                x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)
            iteration += 1
            if max_pki_index in adding_points:
                iteration_of_max_pki = iteration
                break

    return iteration_of_max_pki, max_pki

In [95]:
regr_search = {}
for dataset in perfect_model_regr.keys():
    if dataset not in regr_search.keys():
        regr_search[dataset] = search_of_max_pki_in_dataset_regr(dataset, perfect_model_regr)

In [112]:
with open ('new_results/perfect_regr_search_of_max_pki_in_int_test.pickle', 'wb') as f:
    pickle.dump(regr_search, f)

In [32]:
with open ('new_results/perfect_class_search_of_max_pki_in_int_test.pickle', 'wb') as f:
    pickle.dump(class_search, f)

In [5]:
def search_of_max_pki_rank_in_test_class(dataset, perfect_model):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_r_test_ex, y_train_ex, y_test_ex = data[0], data[1], data[3], data[4], data[5]   
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    while all(x in x_test_in.index for x in perfect_model[dataset][1]) != True:
        x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    max_pki_index_test = y_r_test_ex.idxmax()[0]
    max_pki_test = y_r_test_ex.max()[0]
    best_points = perfect_model[dataset][1][:]
    rank_of_max_pki = dict()
    iteration = 0
    iteration_of_max_pki = 'not found'
    while len(best_points) > 0:
        rfc = RandomForestClassifier(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
        rfc.fit(x_train_in.values, y_train_in['activity'].values)
        classes = set(y_train_in['activity'].values)

        adding_points = best_points[:5]
        del best_points[:5]
        for point in adding_points:
            x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
            x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)
        pred = rfc.predict(x_test_ex.values)
        proba = rfc.predict_proba(x_test_ex.values)
        if len(classes) == 1:
            if 1 in classes:
                proba = np.insert(proba, 0, 0, axis = 1)
            else:
                proba = np.insert(proba, 1, 0, axis = 1)

        prob_of_label_1_test = []
        for n, prob in enumerate(proba):
            prob_of_label_1_test.append([x_test_ex.index[n], prob[1]])
        
        
        ordered_probs = dict()
        ranked_probs = rankdata(prob_of_label_1_test, method='dense', axis=0)
        max_rank = np.amax(ranked_probs, axis = 0)[1]
        for i in ranked_probs:
            i[1] = max_rank - i[1]
        for n, el in enumerate(prob_of_label_1_test):
            ordered_probs[el[0]] = ranked_probs[n][1]
        rank_of_max_pki[iteration] = ordered_probs[max_pki_index_test]
        
        if iteration_of_max_pki == 'not found':
            number_of_top = 0
            top_indexes = []
            i = 0
            while number_of_top<5:
                number_of_top += list(ordered_probs.values()).count(i)
                for index, rank in ordered_probs.items():
                    if rank == i:
                        top_indexes.append(index)
                i+=1
            if max_pki_index_test in top_indexes:
                iteration_of_max_pki = iteration 
            else:
                pass
                
        iteration += 1
        
    return rank_of_max_pki, iteration_of_max_pki, max_pki_test

In [8]:
def search_of_max_pki_rank_in_test_regr(dataset, perfect_model):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_train_ex, y_test_ex = data[0], data[1], data[2], data[3]
    
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    while all(x in x_test_in.index for x in perfect_model[dataset][1]) != True:
        x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    max_pki_index_test = y_test_ex.idxmax()[0]
    max_pki_test = y_test_ex.max()[0]
    best_points = perfect_model[dataset][1][:]
    rank_of_max_pki = dict()
    iteration = 0
    iteration_of_max_pki = 'not found'
    while len(best_points) > 0:
        rfr = RandomForestRegressor(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
        rfr.fit(x_train_in.values, y_train_in['pKi'].values)

        adding_points = best_points[:5]
        del best_points[:5]
        for point in adding_points:
            x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
            x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)
       
        pred_test = rfr.predict(x_test_ex.values)
        pred_value_test = []
        for n, pred in enumerate(pred_test):
            pred_value_test.append([x_test_ex.index[n], pred])
              
        ordered_preds = dict()
        ranked_preds = rankdata(pred_value_test, method='dense', axis=0)
        max_rank = np.amax(ranked_preds, axis = 0)[1]
        for i in ranked_preds:
            i[1] = max_rank - i[1]
        for n, el in enumerate(pred_value_test):
            ordered_preds[el[0]] = ranked_preds[n][1]
        rank_of_max_pki[iteration] = ordered_preds[max_pki_index_test]
        
        if iteration_of_max_pki == 'not found':
            number_of_top = 0
            top_indexes = []
            i = 0
            while number_of_top<5:
                number_of_top += list(ordered_preds.values()).count(i)
                for index, rank in ordered_preds.items():
                    if rank == i:
                        top_indexes.append(index)
                i+=1
            if max_pki_index_test in top_indexes:
                iteration_of_max_pki = iteration 
            else:
                pass

        iteration += 1
        
    return rank_of_max_pki, iteration_of_max_pki, max_pki_test

In [13]:
rank_class = {}
for dataset in tqdm(perfect_model_class.keys()):
    if dataset not in rank_class:
        rank_class[dataset] = search_of_max_pki_rank_in_test_class(dataset, perfect_model_class)

  0%|          | 0/17 [00:00<?, ?it/s]

0 51
not found
1 33
not found
2 21
not found
3 32
not found
4 29
not found
5 30
not found
6 33
not found
7 33
not found
8 32
not found
9 36
not found
10 38
not found
11 37
not found
12 37
not found
13 36
not found
14 38
not found
15 42
not found
16 41
not found
17 40
not found
18 42
not found
19 38
not found
20 41
not found
21 39
not found
22 37
not found
23 40
not found
24 38
not found
25 37
not found
26 38
not found
27 42
not found
28 40
not found


 41%|████      | 7/17 [01:07<01:35,  9.58s/it]

29 40
not found
0 49
not found
1 53
not found
2 65
not found
3 47
not found
4 50
not found
5 44
not found
6 16
not found
7 27
not found
8 22
not found
9 8
not found
10 9
not found
11 12
not found
12 13
not found
13 17
not found
14 17
not found
15 18
not found
16 22
not found
17 18
not found
18 29
not found
19 21
not found
20 25
not found
21 23
not found
22 16
not found
23 16
not found
24 13
not found
25 13
not found
26 12
not found
27 15
not found
28 15
not found


 47%|████▋     | 8/17 [02:12<03:58, 26.48s/it]

...

29 3
26
0 0
0
1 0
0
2 0
0
3 1
0
4 1
0
5 0
0
6 0
0
7 0
0
8 0
0
9 0
0
10 0
0
11 0
0
12 0
0
13 0
0
14 0
0
15 0
0
16 0
0
17 0
0
18 0
0
19 0
0
20 0
0
21 0
0
22 0
0
23 1
0
24 0
0
25 0
0
26 0
0
27 0
0
28 0
0


 94%|█████████▍| 16/17 [11:29<01:07, 67.17s/it]

29 0
0
0 35
not found
1 38
not found
2 37
not found
3 40
not found
4 45
not found
5 42
not found
6 37
not found
7 36
not found
8 46
not found
9 39
not found
10 43
not found
11 45
not found
12 47
not found
13 51
not found
14 52
not found
15 52
not found
16 50
not found
17 43
not found
18 61
not found
19 63
not found
20 64
not found
21 63
not found
22 56
not found
23 56
not found
24 66
not found
25 64
not found
26 62
not found
27 74
not found
28 77
not found


100%|██████████| 17/17 [12:35<00:00, 44.45s/it]

29 62
not found


In [15]:
with open('new_results/perfect_rank_class', 'wb') as f:
    pickle.dump(rank_class, f)

In [103]:
rank_regr = {}
for dataset in tqdm(perfect_model_regr.keys()):
    if dataset not in rank_regr.keys():
        rank_regr[dataset] = search_of_max_pki_rank_in_test_regr(dataset, perfect_model_regr)

  0%|          | 0/20 [00:00<?, ?it/s]

0 92
1 86
2 128
3 56
4 42
5 42
6 42
7 42
8 42
9 41
10 38
11 37
12 35
13 40
14 40
15 40
16 41
17 40
18 38
19 38
20 41
21 38
22 40
23 41
24 40
25 38
26 41
27 36
28 39


  5%|▌         | 1/20 [00:53<16:48, 53.10s/it]

29 39
0 6
1 14
2 12
3 18
4 16
5 19
6 20
7 20
8 18
9 18
10 18
11 19
12 21
13 19
14 21
15 20
16 19
17 23
18 22
19 26
20 26
21 16
22 10
23 17
24 11
25 16
26 13
27 16
28 18


 10%|█         | 2/20 [01:46<15:55, 53.10s/it]

...

29 80
0 72
1 80
2 42
3 49
4 43
5 32
6 50
7 25
8 18
9 22
10 24
11 19
12 25
13 18
14 34
15 39
16 34
17 28
18 24
19 22
20 25
21 24
22 28
23 22
24 29
25 32
26 37
27 43
28 32


 95%|█████████▌| 19/20 [05:20<00:45, 45.80s/it]

29 33
0 36
1 55
2 53
3 56
4 58
5 61
6 61
7 60
8 50
9 42
10 41
11 31
12 39
13 35
14 40
15 30
16 38
17 36
18 39
19 44
20 43
21 42
22 48
23 42
24 39
25 50
26 45
27 43
28 40


100%|██████████| 20/20 [06:16<00:00, 18.82s/it]

29 44


In [110]:
with open('new_results/perfect_rank_regr', 'wb') as f:
    pickle.dump(rank_regr, f)

In [212]:
def EF_regression(dataset, perfect_model):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_train_ex, y_test_ex = data[0], data[1], data[2], data[3]
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    
    while all(x in x_test_in.index for x in perfect_model[dataset][1]) != True:
        x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
       
    results = []
    iteration = 0
    best_points = perfect_model[dataset][1][:]
    
    while len(best_points) > 0:
        
        rfr = RandomForestRegressor(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
        rfr.fit(x_train_in.values, y_train_in['pKi'].values)
 
         
        adding_points = best_points[:5]
        del best_points[:5]
        for point in adding_points:
            x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
            x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)

        pred_test = rfr.predict(x_test_ex.values)
        pred_value_test = []
        for n, pred in enumerate(pred_test):
            pred_value_test.append([x_test_ex.index[n], pred])
       
        ordered_preds = dict()
        ranked_preds = rankdata(pred_value_test, method='dense', axis=0)
        max_rank = np.amax(ranked_preds, axis = 0)[1]
        for i in ranked_preds:
            i[1] = max_rank - i[1]
        for n, el in enumerate(pred_value_test):
            ordered_preds[el[0]] = ranked_preds[n][1]
        
     
        number_of_points = 0
        for rank in range (max(list(ordered_preds.values()))):
            while number_of_points < round(y_test_ex.shape[0] * 0.1):
                number_of_points += list(ordered_preds.values()).count(rank)
                max_rank = rank 
                break
      
        predicted_max_pki = set()
        for key, item in ordered_preds.items():
            if item <= max_rank:
                predicted_max_pki.add(key)
       
        true_max_pki = set(y_test_ex.sort_values(by = 'pKi', ascending=False).index[:number_of_points])
       
        try:
            results.append([iteration, len(true_max_pki.intersection(predicted_max_pki))/number_of_points, number_of_points/y_test_ex.shape[0]])
        except ZeroDivisionError:
            results.append([iteration, 0, number_of_points/y_test_ex.shape[0]])

        iteration += 1 

    return results

In [64]:
def EF_classification(dataset, perfect_model):
    data = []  
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))

    x_train_ex, x_test_ex, y_test_ex_regr, y_train_ex, y_test_ex = data[0], data[1], data[3], data[4], data[5]
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    while all(x in x_test_in.index for x in perfect_model[dataset][1]) != True:
        x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
       
    results = []
    iteration = 0
    best_points = perfect_model[dataset][1][:]
    while len(best_points) > 0:
        
        rfc = RandomForestClassifier(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
        rfc.fit(x_train_in.values, y_train_in['activity'].values)
        classes = set(y_train_in['activity'].values)
 
        adding_points = best_points[:5]
        del best_points[:5]
        
        for point in adding_points:
            x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
            x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)

        pred_ex = rfc.predict(x_test_ex.values)
        proba_ex = rfc.predict_proba(x_test_ex.values)
        if len(classes) == 1:
            if 1 in classes:
                proba_ex = np.insert(proba_ex, 0, 0, axis = 1)
            else:
                proba_ex = np.insert(proba_ex, 1, 0, axis = 1)

        prob_of_label_1_test = []
        for n, prob in enumerate(proba_ex):
            prob_of_label_1_test.append([x_test_ex.index[n], prob[1]])

       
        ordered_probs = dict()
        ranked_probs = rankdata(prob_of_label_1_test, method='dense', axis=0)
        max_rank = np.amax(ranked_probs, axis = 0)[1]
        for i in ranked_probs:
            i[1] = max_rank - i[1]
       
        for n, el in enumerate(prob_of_label_1_test):
            ordered_probs[el[0]] = ranked_probs[n][1]
     
        number_of_points = 0
        for rank in range (max(list(ordered_probs.values()))):
            while number_of_points < round(y_test_ex.shape[0] * 0.1):
                number_of_points += list(ordered_probs.values()).count(rank)
                max_rank = rank 
                break
      
        predicted_max_pki = set()
        for key, item in ordered_probs.items():
            if item <= max_rank:
                predicted_max_pki.add(key)
       
        true_max_pki = set(y_test_ex_regr.sort_values(by = 'pKi', ascending=False).index[:number_of_points])
       
        try:
            results.append([iteration, len(true_max_pki.intersection(predicted_max_pki))/number_of_points, number_of_points/y_test_ex.shape[0]])
        except ZeroDivisionError:
            results.append([iteration, 0, number_of_points/y_test_ex.shape[0]])
        
        iteration += 1 
    return results

In [ ]:
EF_regr = {}
for dataset in tqdm(perfect_model_regr.keys()):    
    EF_regr[dataset] = EF_regression(dataset, perfect_model_regr)

In [65]:
EF_class = {}
for dataset in tqdm(perfect_model_class.keys()):    
    EF_class[dataset] = EF_classification(dataset, perfect_model_class)

100%|██████████| 17/17 [18:11<00:00, 64.20s/it]


In [214]:
with open('new_results/perfect_EF_regr', 'wb') as f:
    pickle.dump(EF_regr, f)

In [67]:
with open('new_results/perfect_EF_class', 'wb') as f:
    pickle.dump(EF_class, f)

In [21]:
def AUC_10(dataset, perfect_model):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_test_ex_regr, y_train_ex, y_test_ex = data[0], data[1], data[3], data[4], data[5]
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    while all(x in x_test_in.index for x in perfect_model[dataset][1]) != True:
        x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    number_of_points = round(y_test_ex.shape[0] * 0.1)
    results = []
    iteration = 0
    best_points = perfect_model[dataset][1][:]
    while len(best_points) > 0:
    
        
        rfc = RandomForestClassifier(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
        rfc.fit(x_train_in.values, y_train_in['activity'].values)
        classes = set(y_train_in['activity'].values)
 
        adding_points = best_points[:5]
        del best_points[:5]
        
        for point in adding_points:
            x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
            x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)

        pred_ex = rfc.predict(x_test_ex.values)
        proba_ex = rfc.predict_proba(x_test_ex.values)
        if len(classes) == 1:
            if 1 in classes:
                proba_ex = np.insert(proba_ex, 0, 0, axis = 1)
            else:
                proba_ex = np.insert(proba_ex, 1, 0, axis = 1)
        max_pkis = set(y_test_ex_regr.sort_values(by = 'pKi', ascending=False).index[:number_of_points])
        for value in y_test_ex.index:
            if value in max_pkis:
                y_test_ex.loc[value]['activity'] = 1 
            else:
                y_test_ex.loc[value]['activity'] = 0
       
        fpr, tpr, _ = roc_curve(y_test_ex.values, [x[1] for x in proba_ex])
       
        AUC = auc(fpr, tpr)
        results.append([iteration, AUC])
        
        iteration += 1 

    return results

In [22]:
AUC10_class = {}
for dataset in tqdm(perfect_model_class.keys()):
    AUC10_class[dataset] = AUC_10(dataset, perfect_model_class)

  0%|          | 0/17 [00:00<?, ?it/s]

[0, 0.5625]
[1, 0.5606617647058824]
[2, 0.521139705882353]
[3, 0.8060661764705884]
[4, 0.8621323529411765]
[5, 0.8998161764705883]
[6, 0.9145220588235294]
[7, 0.8988970588235294]
[8, 0.8841911764705882]
[9, 0.8860294117647058]
[10, 0.8970588235294118]
[11, 0.9044117647058824]
[12, 0.8869485294117647]
[13, 0.8970588235294117]
[14, 0.9181985294117647]
[15, 0.9016544117647058]
[16, 0.8887867647058822]
[17, 0.8262867647058824]
[18, 0.8290441176470589]
[19, 0.84375]
[20, 0.8694852941176471]
[21, 0.8566176470588236]
[22, 0.8851102941176471]
[23, 0.9053308823529411]
[24, 0.9117647058823529]
[25, 0.9080882352941176]
[26, 0.8933823529411764]
[27, 0.8970588235294117]
[28, 0.9025735294117647]


  6%|▌         | 1/17 [01:06<17:47, 66.72s/it]

[29, 0.9172794117647058]
[0, 0.5]
[1, 0.5]
[2, 0.45228758169934635]
[3, 0.3928104575163398]
[4, 0.34901960784313724]
[5, 0.3627450980392156]
[6, 0.36732026143790847]
[7, 0.3699346405228758]
[8, 0.4176470588235294]
[9, 0.4405228758169935]
[10, 0.45424836601307195]
[11, 0.4673202614379085]
[12, 0.4941176470588235]
[13, 0.4797385620915033]
[14, 0.45686274509803926]
[15, 0.469281045751634]
[16, 0.5725490196078431]
[17, 0.5640522875816993]
[18, 0.5673202614379085]
[19, 0.5176470588235293]
[20, 0.5333333333333333]
[21, 0.515032679738562]
[22, 0.5183006535947712]
[23, 0.5392156862745098]
[24, 0.5405228758169934]
[25, 0.5444444444444444]
[26, 0.5483660130718954]
[27, 0.5424836601307189]
[28, 0.5274509803921569]


 12%|█▏        | 2/17 [02:12<16:36, 66.45s/it]

...

[29, 0.7747126436781608]
[0, 0.5]
[1, 0.6993087557603687]
[2, 0.7085253456221198]
[3, 0.653225806451613]
[4, 0.7154377880184332]
[5, 0.6970046082949309]
[6, 0.6728110599078342]
[7, 0.6658986175115207]
[8, 0.6705069124423962]
[9, 0.6981566820276497]
[10, 0.7142857142857142]
[11, 0.7039170506912442]
[12, 0.6797235023041475]
[13, 0.7050691244239631]
[14, 0.6923963133640553]
[15, 0.7165898617511521]
[16, 0.7269585253456221]
[17, 0.6923963133640553]
[18, 0.7223502304147466]
[19, 0.7476958525345622]
[20, 0.7569124423963133]
[21, 0.7776497695852534]
[22, 0.7684331797235023]
[23, 0.7788018433179724]
[24, 0.7960829493087558]
[25, 0.8237327188940092]
[26, 0.8225806451612903]
[27, 0.8283410138248848]
[28, 0.8110599078341014]


 94%|█████████▍| 16/17 [18:04<01:08, 68.11s/it]

[29, 0.8133640552995391]
[0, 0.5850558659217877]
[1, 0.5465083798882682]
[2, 0.5484636871508379]
[3, 0.6191340782122905]
[4, 0.6537709497206704]
[5, 0.6236033519553073]
[6, 0.6304469273743016]
[7, 0.630586592178771]
[8, 0.6245810055865921]
[9, 0.6603351955307263]
[10, 0.6335195530726256]
[11, 0.6424581005586592]
[12, 0.604050279329609]
[13, 0.6016759776536312]
[14, 0.614804469273743]
[15, 0.6180167597765364]
[16, 0.6430167597765364]
[17, 0.6451117318435755]
[18, 0.6423184357541899]
[19, 0.6092178770949721]
[20, 0.6305865921787709]
[21, 0.6395251396648045]
[22, 0.6451117318435754]
[23, 0.6508379888268156]
[24, 0.6409217877094971]
[25, 0.6339385474860335]
[26, 0.6456703910614525]
[27, 0.654608938547486]
[28, 0.6713687150837988]


100%|██████████| 17/17 [19:12<00:00, 67.78s/it]

[29, 0.6861731843575418]


In [23]:
with open('new_results/perfect_AUC10_class', 'wb') as f:
    pickle.dump(AUC10_class, f)